# LDA to staticstics

In [62]:
import numpy as np
from sklearn.metrics import roc_curve
def compute_eer(pos_scores, neg_scores):
    score_vector = np.concatenate([pos_scores, neg_scores])
    label_vector = np.concatenate([np.ones(len(pos_scores)), np.zeros(len(neg_scores))])
    fpr, tpr, thres = roc_curve(label_vector, score_vector, pos_label=1)
    eer = np.min([fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))],
                 1-tpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]])
    thres = thres[np.nanargmin(np.abs(fpr - (1 - tpr)))]

    return eer

In [ ]:
import pandas as pd

In [ ]:
trial = pd.read_csv("../sv_system/datasets/voxceleb1/dataframes/voxc1_sv_trial.csv")

In [107]:
import pickle 

voxc1_si_stats, voxc1_si_labels = pickle.load(open("../sv_system/embeddings/voxc1_fbank64_voxc12_stat/voxc1_si_stat.pkl", "rb"))
voxc1_sv_stats, voxc1_sv_labels = pickle.load(open("../sv_system/embeddings/voxc1_fbank64_voxc12_stat/voxc1_sv_stat.pkl", "rb"))

In [108]:
voxc1_si_mu, voxc1_si_std = voxc1_si_stats[:, :1500], voxc1_si_stats[:, 1500:]
voxc1_sv_mu, voxc1_sv_std = voxc1_sv_stats[:, :1500], voxc1_sv_stats[:, 1500:]

In [122]:
voxc1_si_stat = voxc1_si_std
voxc1_sv_stat = voxc1_sv_std

In [123]:
voxc1_si_stat = voxc1_si_stat
voxc1_sv_stat = voxc1_sv_stat
# voxc1_si_mean = voxc1_si_stat.mean(0, keepdim=True)
# voxc1_si_stat = voxc1_si_stat - voxc1_si_mean
# voxc1_sv_stat = voxc1_sv_stat - voxc1_si_mean

In [124]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

clf = LDA(solver='svd', n_components=200)
clf.fit(voxc1_si_stat, voxc1_si_labels)

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=200, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [125]:
import torch
lda_voxc1_sv_stat = torch.from_numpy(clf.transform(voxc1_sv_stat))

In [126]:
from torch.nn.functional import cosine_similarity as cosine
lda_score = cosine(lda_voxc1_sv_stat[trial.enroll_idx], lda_voxc1_sv_stat[trial.test_idx], dim=1)
cos_eer = compute_eer(lda_score.numpy()[trial.label==1], lda_score.numpy()[trial.label==0])

In [128]:
from torch.nn.functional import cosine_similarity as cosine
score = cosine(voxc1_sv_stat[trial.enroll_idx], voxc1_sv_stat[trial.test_idx], dim=1)
cos_eer = compute_eer(score.numpy()[trial.label==1], score.numpy()[trial.label==0])

In [130]:
# mu + std
cos_eer

0.2127253446447508

In [131]:
# mu
cos_eer

0.2127253446447508

In [132]:
# std
cos_eer

0.2127253446447508